In [ ]:
import lance

r = lance.dataset("/home/maximilien/work/bop_data/bop_ds.lance/")

In [ ]:
for batch in r.to_batches(limit=10):
    a = batch.to_pandas()
    break

In [ ]:
import duckdb

duckdb.query("SELECT id, category_id FROM a").to_df()

In [ ]:
import json
import pandas as pd

with open("/home/maximilien/work/adapt-2023_with_gt/info.json", "r") as json_file:
    info = json.load(json_file)

with open(
    "/home/maximilien/work/adapt-2023_with_gt/test_targets_adapt23.json", "r"
) as json_file:
    data = json.load(json_file)


df = pd.DataFrame(data)
info_df = pd.DataFrame(info)

# Renommer la colonne "obj_id" en "category_id"
df.rename(columns={"obj_id": "category_id"}, inplace=True)


result_table = df.groupby(["category_id"])["scene_id"].agg(set).reset_index()

# Renommer la colonne "scene_id" en "scenes_ids"
result_table.rename(columns={"scene_id": "scenes_ids"}, inplace=True)

# Compter le nombre d'occurrences
result_table["number of occurrences"] = (
    df.groupby(["category_id"])["scene_id"].size().values
)

result_table["category_id"] = result_table["category_id"].apply(
    lambda i: info["category_index"][str(i)]
)

result_table = (
    result_table.set_index(["category_id"])
    .join(info_df.set_index(["category_index"]), how="inner")
    .reset_index()
)

result_table = result_table.get(
    ["index", "scenes_ids", "number of occurrences", "symmetries", "bbox"]
)

In [ ]:
# Fonction pour calculer le diamètre et le size_ratio à partir des valeurs bbox
def calculate_size(bbox):
    diameter = (bbox[0] ** 2 + bbox[1] ** 2 + bbox[2] ** 2) ** 0.5
    size_ratio = min(bbox) / max(bbox)
    return diameter, size_ratio


result_table[["diameter", "size_ratio"]] = result_table["bbox"].apply(
    lambda bbox: pd.Series(calculate_size(bbox))
)

result_table.rename(columns={"bbox": "bbox (mm)"}, inplace=True)

result_table